# Dataset Covid-19 no Brasil
Fonte:
- https://brasil.io/dataset/covid19/caso_full/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

Import do dataframe

In [3]:
df = pd.read_csv('/content/drive/MyDrive/data_vis/caso_full.csv')

In [4]:
df

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-17,0.0000,0,1,city,AC,3,0
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0000,0,1,state,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-18,0.0000,0,2,city,AC,0,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0000,0,2,state,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,False,4,0.96754,2020-03-19,0.0000,0,3,city,AC,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837499,Tupirama,1721257.0,2021-04-02,202113,1922.0,1891.0,False,True,127,6607.70031,2021-03-31,0.0157,2,315,city,TO,0,0
1837500,Tupiratins,1721307.0,2021-04-02,202113,2729.0,2671.0,False,True,51,1868.81642,2021-03-31,0.0196,1,315,city,TO,0,0
1837501,Wanderlândia,1722081.0,2021-04-02,202113,11734.0,11683.0,False,True,662,5641.72490,2021-03-31,0.0121,8,340,city,TO,0,0
1837502,Xambioá,1722107.0,2021-04-02,202113,11520.0,11540.0,False,True,1618,14045.13889,2021-03-31,0.0117,19,329,city,TO,0,0


In [5]:
df_per_state = df.loc[lambda df: df['place_type'] == 'state']

In [6]:
df_dataset = df_per_state.loc[lambda df: df['state'] == 'DF']

In [7]:
!pip install -U plotly
# Using graph_objects and express plotly
import plotly.graph_objects as go
import plotly.express as px

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


## Scatter plot e line plot

### Distrito Federal
Nossa primeira visualização é um scatter plot mapeando a evolução do número de óbitos pelo tempo no Distrito Federal.

In [8]:
fig = go.Figure([go.Scatter(x=df_dataset['last_available_date'], y=df_dataset['last_available_deaths'])])
fig.show()

Essa mesma ideia agora usando um gráfico de barras. É o que costumamos ver nos jornais.

In [9]:
fig = px.bar(df_dataset, x=df_dataset.last_available_date, y=df_dataset.last_available_deaths)
fig.show()

### Comparação dos números por regiões
Visualização da evolução do número de casos por região. Primeiro, precisamos construir um mapeamento de estados/regiões.

In [10]:
region_dict = {
    'AC': 'Norte',
    'AP': 'Norte',
    'AM': 'Norte',
    'PA': 'Norte',
    'RO': 'Norte',
    'RR': 'Norte',
    'TO': 'Norte',
    'AL': 'Nordeste',
    'BA': 'Nordeste',
    'CE': 'Nordeste',
    'MA': 'Nordeste',
    'PB': 'Nordeste',
    'PE': 'Nordeste',
    'PI': 'Nordeste',
    'RN': 'Nordeste',
    'SE': 'Nordeste',
    'DF': 'Centro-oeste',
    'GO': 'Centro-oeste',
    'MT': 'Centro-oeste',
    'MS': 'Centro-oeste',
    'ES': 'Sudeste',
    'MG': 'Sudeste',
    'RJ': 'Sudeste',
    'SP': 'Sudeste',
    'PR': 'Sul',
    'RS': 'Sul',
    'SC': 'Sul'
}

In [11]:
df['region'] = df.state.replace(region_dict)
df_per_state['region'] = df_per_state.state.replace(region_dict)
df_per_state

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths,region
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0000,0,1,state,AC,3,0,Norte
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0000,0,2,state,AC,0,0,Norte
5,NaN,12.0,2020-03-19,202012,894470.0,881935.0,False,False,4,0.44719,2020-03-19,0.0000,0,3,state,AC,1,0,Norte
7,NaN,12.0,2020-03-20,202012,894470.0,881935.0,False,False,7,0.78259,2020-03-20,0.0000,0,4,state,AC,3,0,Norte
9,NaN,12.0,2020-03-21,202012,894470.0,881935.0,False,False,11,1.22978,2020-03-21,0.0000,0,5,state,AC,4,0,Norte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836943,NaN,17.0,2021-03-29,202113,1590248.0,1572866.0,False,False,140018,8804.79020,2021-03-29,0.0143,2007,377,state,TO,924,23,Norte
1837083,NaN,17.0,2021-03-30,202113,1590248.0,1572866.0,False,False,140975,8864.96949,2021-03-30,0.0144,2032,378,state,TO,957,25,Norte
1837223,NaN,17.0,2021-03-31,202113,1590248.0,1572866.0,False,False,141913,8923.95400,2021-03-31,0.0145,2057,379,state,TO,938,25,Norte
1837363,NaN,17.0,2021-04-01,202113,1590248.0,1572866.0,True,False,142493,8960.42630,2021-04-01,0.0146,2079,380,state,TO,580,22,Norte


Os plots serão feitos com line, que é o tipo mais legível para comparações como essas.

In [12]:
df_sudeste = df_per_state.query("region == 'Sudeste'")
fig = px.line(df_sudeste, x='last_available_date', y='last_available_deaths', color='state')
fig.show()

In [13]:
df_nordeste = df_per_state.query("region == 'Nordeste'")
fig = px.line(df_nordeste, x='last_available_date', y='last_available_deaths', color='state')
fig.show()

In [14]:
df_co = df_per_state.query("region == 'Centro-oeste'")
fig = px.line(df_co, x='last_available_date', y='last_available_deaths', color='state')
fig.show()

In [15]:
df_norte = df_per_state.query("region == 'Norte'")
fig = px.line(df_norte, x='last_available_date', y='last_available_deaths', color='state')
fig.show()

In [16]:
df_sul = df_per_state.query("region == 'Sul'")
fig = px.line(df_sul, x='last_available_date', y='last_available_deaths', color='state')
fig.show()

## Bubble chart
Agora vamos para um plot mais elaborado - #adaptação_descarada_da_documentação. Vamos fazer um bubble chart relacionando número de óbitos e número de casos.

Queremos encontrar, para todos os estados, a data mais recente em que todos tiveram leituras. Como a coluna `last_available_date` está no formato datetime, podemos usar a função max que retorna exatamente o queremos. Uma boa referência nesse intuito são os seguintes links:
- https://towardsdatascience.com/working-with-datetime-in-pandas-dataframe-663f7af6c587
- https://stackoverflow.com/questions/29886609/find-most-recent-date-in-pandas-dataframe

In [17]:
most_recent_date = df_per_state['last_available_date'].max()
most_recent_date

'2021-04-02'

Agora que temos a data comum mais recente, basta selecionar no dataframe as linhas com essa data.

In [18]:
df_per_state.loc[df_per_state['last_available_date'] == '2021-04-02']

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths,region
7892,NaN,12.0,2021-04-02,202113,894470.0,881935.0,True,False,70851,7921.00350,2021-04-02,0.0181,1280,382,state,AC,863,11,Norte
42676,NaN,27.0,2021-04-02,202113,3351543.0,3337357.0,True,False,154819,4619.33503,2021-04-02,0.0232,3598,391,state,AL,620,22,Nordeste
64679,NaN,13.0,2021-04-02,202113,4207714.0,4144597.0,True,False,351118,8344.62608,2021-04-02,0.0343,12048,386,state,AM,830,17,Norte
70637,NaN,16.0,2021-04-02,202113,861773.0,845731.0,True,False,98324,11409.50111,2021-04-02,0.0134,1318,379,state,AP,280,7,Norte
207974,NaN,29.0,2021-04-02,202113,14930634.0,14873064.0,True,False,811808,5437.19711,2021-04-02,0.0192,15599,393,state,BA,3347,127,Nordeste
272934,NaN,23.0,2021-04-02,202113,9187103.0,9132078.0,True,False,547414,5958.50509,2021-04-02,0.0261,14305,383,state,CE,3660,37,Nordeste
274091,NaN,53.0,2021-04-02,202113,3055149.0,3015268.0,True,False,346873,11353.71794,2021-04-02,0.0179,6207,392,state,DF,1191,57,Centro-oeste
302325,NaN,32.0,2021-04-02,202113,4064052.0,4018650.0,True,False,386219,9503.29868,2021-04-02,0.0197,7604,394,state,ES,1745,46,Sudeste
380322,NaN,52.0,2021-04-02,202113,7113540.0,7018354.0,True,False,488199,6862.95431,2021-04-02,0.0242,11802,387,state,GO,851,90,Centro-oeste
453648,NaN,21.0,2021-04-02,202113,7114598.0,7075181.0,True,False,244340,3434.34724,2021-04-02,0.0252,6149,379,state,MA,620,41,Nordeste


Tendo todos os dados necessários no dataframe, vamos ao plot. A referência foi o link https://plotly.com/python/bubble-charts/#categorical-bubble-charts.

Vamos filtrar o dataframe com o valor mais recente da semana epidemiológica, que é uma contagem semana-a-semana da pandemia.

In [19]:
import math

# Load data, define hover text and bubble size
#df_bubble = df_per_state[df_per_state['epidemiological_week']==202113]
df_bubble = df_per_state.loc[df_per_state['last_available_date'] == '2021-04-02']
df_bubble = df_bubble.sort_values(['region', 'state'])

hover_text = []
bubble_size = []

for index, row in df_bubble.iterrows():
    hover_text.append(('Estado: {state}<br>'+
                      'Casos confirmados: {cases}<br>'+
                      'Óbitos confirmados: {deaths}<br>'+
                      'Population: {pop}<br>'+
                      'Semana epidemiológica: {week}').format(state=row['state'],
                                            cases=row['last_available_confirmed'],
                                            deaths=row['last_available_deaths'],
                                            pop=row['estimated_population'],
                                            week=row['epidemiological_week']))
    bubble_size.append(math.sqrt(row['estimated_population']))

df_bubble['text'] = hover_text
df_bubble['size'] = bubble_size
sizeref = 2.*max(df_bubble['size'])/(100**2)

# Dictionary with dataframes for each region
region_names = ['Norte', 'Nordeste', 'Centro-oeste', 'Sudeste', 'Sul']
region_data = {region:df_bubble.query("region == '%s'" %region)
                              for region in region_names}

# Create figure
fig = go.Figure()

for region_name, region in region_data.items():
    fig.add_trace(go.Scatter(
        x=region['last_available_confirmed'], y=region['last_available_deaths'],
        name=region_name, text=region['text'],
        marker_size=region['size'],
        ))

# Tune marker appearance and layout
fig.update_traces(mode='markers', marker=dict(sizemode='area',
                                              sizeref=sizeref, line_width=2))

fig.update_layout(
    title='Casos confirmados v. Óbitos confirmados, Semana EP 202113',
    xaxis=dict(
        title='Casos confirmados',
        gridcolor='white',
        type='log',
        gridwidth=2,
    ),
    yaxis=dict(
        title='Óbitos confirmados',
        gridcolor='white',
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)
fig.show()

Vamos refazer esse plot, agora levando em conta o índice de mortalidade x casos confirmados por 100k habitantes.

In [20]:
df_bubble = df_per_state.loc[df_per_state['last_available_date'] == '2021-04-02']
df_bubble = df_bubble.sort_values(['region', 'state'])

hover_text = []
bubble_size = []

for index, row in df_bubble.iterrows():
    hover_text.append(('Estado: {state}<br>'+
                      'Casos confirmados/100k habitantes: {cases}<br>'+
                      'Índice de mortalidade: {deaths}<br>'+
                      'Population: {pop}<br>'+
                      'Semana epidemiológica: {week}').format(state=row['state'],
                                            cases=row['last_available_confirmed_per_100k_inhabitants'],
                                            deaths=row['last_available_death_rate'],
                                            pop=row['estimated_population'],
                                            week=row['epidemiological_week']))
    bubble_size.append(math.sqrt(row['estimated_population']))

df_bubble['text'] = hover_text
df_bubble['size'] = bubble_size
sizeref = 2.*max(df_bubble['size'])/(100**2)

# Dictionary with dataframes for each region
region_names = ['Norte', 'Nordeste', 'Centro-oeste', 'Sudeste', 'Sul']
region_data = {region:df_bubble.query("region == '%s'" %region)
                              for region in region_names}

# Create figure
fig = go.Figure()

for region_name, region in region_data.items():
    fig.add_trace(go.Scatter(
        x=region['last_available_confirmed_per_100k_inhabitants'], y=region['last_available_death_rate'],
        name=region_name, text=region['text'],
        marker_size=region['size'],
        ))

# Tune marker appearance and layout
fig.update_traces(mode='markers', marker=dict(sizemode='area',
                                              sizeref=sizeref, line_width=2))

fig.update_layout(
    title='Casos confirmados/100k habitantes v. Índice de Mortalidade, Semana EP 202113',
    xaxis=dict(
        title='Casos confirmados/100k',
        gridcolor='white',
        type='log',
        gridwidth=2,
    ),
    yaxis=dict(
        title='Índice de Mortalidade',
        gridcolor='white',
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)
fig.show()

Quanto mais pra cima no plot, maior é o número de pessoas que morrem de Covid-19 frente à todos os infectados com Covid-19. Quanto mais à direita, maior o número relativo de casos da doença.